# Mistake: z-scoring along the wrong axis

In [1]:
import numpy as np
from numpy import shape, matrix, log, exp, zeros,random,dot,multiply
from mnist import readWithoutBias

In [2]:
data_train, label_train = readWithoutBias(dataset="training")
data_test, label_test = readWithoutBias(dataset="testing")

In [3]:
label_train = matrix(label_train)
label_test  = matrix(label_test)

In [5]:
shape(data_train),shape(label_train)

((60000, 784), (60000, 1))

##Code with only One input

In [118]:
alpha = 0.001
n_input = shape(data_train)[1] # excluding bias term
n_hidden = 20                  # excluding bias term
n_output = 10
epsilon = 0.7
random.seed(0)
theta1 = matrix(random.rand(n_hidden,n_input+1)*2*epsilon - epsilon)
theta2 = matrix(random.rand(n_output,n_hidden+1)*2*epsilon - epsilon)
#shape(theta1),shape(theta2)
#((20, 785), (10, 21))

In [119]:

X = matrix(data_train[0,:]).T
y = label_train[0]

####Forward prop

In [120]:
X = (X - np.mean(X))/np.std(X)
a1 = np.vstack(([1],X))
z2 = dot(theta1,a1)
a2 = np.vstack(([1],sigmoid(z2)))
z3 = dot(theta2,a2)
a3 = sigmoid(z3)

#### Back prop

In [127]:
t = np.zeros_like(a3)
t[y] = 1
delta3 = a3-t
delta2 = dot(theta2.T,delta3)[1:]
delta2 = multiply(delta2,sigmoidGradient(z2))

gradTheta2 = dot(delta3,a2.T)
gradTheta1 = dot(delta2,a1.T)

In [145]:
shape(numGrad1-gradTheta1),shape(numGrad2),shape(gradTheta2), shape(gradTheta1), type(t)

((20, 785), (10, 21), (10, 21), (20, 785), numpy.matrixlib.defmatrix.matrix)

In [125]:
a3

matrix([[ 0.47986079],
        [ 0.2614446 ],
        [ 0.76381289],
        [ 0.21341023],
        [ 0.64900345],
        [ 0.31705368],
        [ 0.55712291],
        [ 0.44706651],
        [ 0.21914671],
        [ 0.27258546]])

In [143]:
gradTheta1, gradTheta2 = backPropGradient(X,y,theta1,theta2)
numGrad1,numGrad2      = numericalGradient(X,y,theta1,theta2)

In [147]:
np.sum(np.subtract(numGrad2,gradTheta2))

5.0095653235442588e-09

### helperfunctions

In [142]:
def numericalGradient(X, y, theta1, theta2):
    epsilon = 10**-4
    numGrad1 = np.zeros_like(theta1)
    numGrad2 = np.zeros_like(theta2)
    for i in range(shape(theta1)[0]):
        for j in range(shape(theta1)[1]):
            theta1_pos = np.copy(theta1)
            theta1_neg = np.copy(theta1)
            theta1_pos[i,j] += epsilon
            theta1_neg[i,j] -= epsilon
            numGrad1[i,j]    = (errorFn(X, y, theta1_pos, theta2) - errorFn(X, y, theta1_neg, theta2))/2/epsilon
    for i in range(shape(theta2)[0]):
        for j in range(shape(theta2)[1]):
            theta2_pos = np.copy(theta2)
            theta2_neg = np.copy(theta2)
            theta2_pos[i,j] += epsilon
            theta2_neg[i,j] -= epsilon
            numGrad2[i,j]    = (errorFn(X, y, theta1, theta2_pos) - errorFn(X, y, theta1, theta2_neg))/2/epsilon
    return numGrad1,numGrad2

In [140]:
def errorFn(X, y, theta1, theta2):
    a1 = np.vstack(([1],X))
    z2 = dot(theta1,a1)
    a2 = np.vstack(([1],sigmoid(z2)))
    z3 = dot(theta2,a2)
    a3 = sigmoid(z3)
    
    t = np.zeros_like(a3)
    t[y] = 1
    
    error = np.sum(multiply(t,log(a3)) + multiply((1-t),log(1-a3)))
    error = -error
    return error

In [130]:
def backPropGradient(X, y, theta1, theta2):
    a1 = np.vstack(([1],X))
    z2 = dot(theta1,a1)
    a2 = np.vstack(([1],sigmoid(z2)))
    z3 = dot(theta2,a2)
    a3 = sigmoid(z3)
    
    t = np.zeros_like(a3)
    t[y] = 1
    
    delta3 = a3-t
    delta2 = dot(theta2.T,delta3)[1:]
    delta2 = multiply(delta2,sigmoidGradient(z2))

    gradTheta2 = dot(delta3,a2.T)
    gradTheta1 = dot(delta2,a1.T)
    return gradTheta1, gradTheta2

In [43]:
def sigmoid(z):
    return 1/(1+exp(-z))
def sigmoidGradient(z):
    s = sigmoid(z)
    return multiply(s,1-s)